### Product Constructor Standings

In [0]:
%run "../includes/configuration"

In [0]:
driver_standings_df = spark.read \
    .parquet(f"{presentation_folder_path}/driver_standings")
display(driver_standings_df)

In [0]:
from pyspark.sql.functions import sum
driver_standings_grouped_df = driver_standings_df.groupBy('race_year', 'team')\
                                .agg(sum('wins').alias('wins'), sum('total_points').alias('total_points'))
display(driver_standings_grouped_df.filter('race_year = 2020'))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))

constructors_rank_df = driver_standings_grouped_df.withColumn('rank', rank().over(constructor_rank_spec))
display(constructors_rank_df.filter('race_year = 2020'))

In [0]:
constructors_rank_df.write\
    .mode('overwrite')\
    .parquet(f'{presentation_folder_path}/constructor_standings')